# Debug （基于python语言)

## 传统的调试姿势
1. print / logging
2. pdb (打断点)
    
### 不好用的地方:
- 适用于开发过程
- 无法持续追踪问题
- 复杂的代码结构进行调试及其困难




## sys.settrace
** 目的: 收集程序运行时的信息。  **


**原理: 基于producer->controller->consumer的模型. **
- producer: 事件的提供者, 也就是我们开发每行代码， 每行代码都会对应一个事件.
- controller: 事件管理。
- consumer: 开发者自己定义， 做一些需要获取信息的处理.

### Quick Strat

In [ ]:
import sys

def foo():
    print("hello world")

def tracer(frame, event, args):
    if frame.f_code.co_filename == 'test.py':
        print({'event': event, 'args': args, 'context': frame.f_code.co_name})
    return tracer
    
if __name__ == '__main__':
    sys.settrace(tracer)
    foo()

    
    {'event': 'call', 'args': None, 'context': 'foo'}
    {'event': 'line', 'args': None, 'context': 'foo'}
    hello world
    {'event': 'return', 'args': None, 'context': 'foo'}
    

#### 事件枚举  
事件 |处理器| 栈帧位置|参数|返回值
 :-| :-| :-|:-|:- 
 call  | global tracer | 函数定义行 | None | local tracer / None
 line  | local tracer | 执行当前行 | None | local tracer / None
 return| local tracer | 返回语句所在行 | 返回值 / None | Ignored
 exception| local tracer| 异常冒泡所在行 | (exc, val, tb) | local tracer / None   
 



#### frame(栈帧)
栈帧就是存储在用户栈上的（当然内核栈同样适用）每一次函数调用涉及的相关信息的记录单元， 栈帧的主要作用是**用来控制和保存一个过程的所有信息的**。

有关python的栈帧数据结构的详细介绍， 可以查看[Python3 inspect module](https://docs.python.org/3/library/inspect.html)

**几个常用的信息:**

属性|保存的信息
:-:| -:
frame.f_lineno | current line number in Python source code
frmae.f_locals | local namespace seen by this frame
frmae.f_code.co_name | name with which this code object was defined
frame.f_code.co_filename | name of file in which this code object was created

### 应用 
#### 变量追踪

In [ ]:
import sys

def quick_sort(vector):
    if len(vector) < 2:
        return vector
    
    else:
        flag = vector[0]
        less = [i for i in vector[1:] if i <= flag]
        big = [i for i in vector[1:] if i > flag]
        final = quick_sort(less) + [flag] + quick_sort(big)
        return final
    
def tracer(frame, event, args):
    if 'final' in frame.f_locals.keys():
        print(f'{frame.f_locals}')
    return tracer

if __name__ == '__main__':
    sys.settrace(tracer)
    test = [1, 9, 2, 4, 1, 2]
    quick_sort(test)

    
    {'vector': [2, 4, 2], 'flag': 2, 'less': [2], 'big': [4], 'final': [2, 2, 4]}
    {'vector': [2, 4, 2], 'flag': 2, 'less': [2], 'big': [4], 'final': [2, 2, 4]}
    {'vector': [9, 2, 4, 2], 'flag': 9, 'less': [2, 4, 2], 'big': [], 'final': [2, 2, 4, 9]}
    {'vector': [9, 2, 4, 2], 'flag': 9, 'less': [2, 4, 2], 'big': [], 'final': [2, 2, 4, 9]}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1, 'less': [1], 'big': [9, 2, 4, 2], 'final': [1, 1, 2, 2, 4, 9]}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1, 'less': [1], 'big': [9, 2, 4, 2], 'final': [1, 1, 2, 2, 4, 9]}

#### 函数过程追踪

In [2]:
import asyncio
import sys

async def func1(future, N):
    await asyncio.sleep(N)
    future.set_result('This is func1')

async def func2(future, N):
    await asyncio.sleep(N)
    future.set_result('This is func2')

def got_result(future):
    print(future.result())

def tracer(frame, event, args):
    if event == 'call' and frame.f_code.co_filename == 'test.py':
        print({'filename': frame.f_code.co_filename,
               'lineno': frame.f_lineno,
               'context': frame.f_code.co_name})
        return tracer
    return tracer

if __name__ == '__main__':
    sys.settrace(tracer)

    loop = asyncio.get_event_loop()
    future1 = asyncio.Future()
    future2 = asyncio.Future()

    task = [asyncio.ensure_future(func1(future1, 3)),
            asyncio.ensure_future(func2(future2, 2))]

    future1.add_done_callback(got_result)
    future2.add_done_callback(got_result)
    loop.run_until_complete(asyncio.wait(task))

    
    {'filename': 'test.py', 'lineno': 5, 'context': 'func1'}
    {'filename': 'test.py', 'lineno': 10, 'context': 'func2'}
    {'filename': 'test.py', 'lineno': 11, 'context': 'func2'}
    {'filename': 'test.py', 'lineno': 15, 'context': 'got_result'}
    This is func2
    {'filename': 'test.py', 'lineno': 6, 'context': 'func1'}
    {'filename': 'test.py', 'lineno': 15, 'context': 'got_result'}
    This is func1

#### 高级应用
python是一门动态语言， 也就没有静态语言类型检查这种优点。代码中经常需要对类型进行判断， assert isinstance(xx, xx), 这种情况就导致可读性比较差。        


为了改善代码可读性的问题， python3.6完善类型注释的语法(pep526)。 facebook根据这套语法开源了一款类型检查的工具: **[pyre-check](https://github.com/facebook/pyre-check)**， 

In [5]:
def hello(name: str) -> str:
    return f"hello, {name}"

print(hello('wang'))
print(hello(11))


hello, wang
hello, 11



    
    ➜  test git:(master) ✗ pyre check
     ƛ Found 1 type error!
    test.py:6:12 Incompatible parameter type [6]: Expected `str` but got `int`.

facebook有百万行级别的代码， 一行一行去添加类型注释不太现实， 所以它们又开发了**[monkeyType](https://github.com/Instagram/MonkeyType)**， 收集程序运行时候的信息，然后将类型信息应用到代码上 

    ➜  test git:(master) ✗ ls      
    main.py test.py


    ➜  test git:(master) ✗ cat test.py
    def hello(name):
        return f'hello, {name}'



    
    ➜  test git:(master) ✗ monkeytype run main.py  && ls          
    main.py            monkeytype.sqlite3 test.py
    

    ➜  test git:(master) ✗ monkeytype apply test && cat test.py
    def hello(name: str) -> str:
        return f'hello, {name}'